In [1]:
import nest_asyncio

nest_asyncio.apply()

# this also required by jupyter to start Isabelle server
import os

original_path = os.environ.get('PATH', '')
new_path = original_path + ':/workspace/Isabelle2023/bin'
os.environ['PATH'] = new_path
print(os.environ['PATH'])

/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/workspace/Isabelle2023/bin


## Hard Critique with GPT-4o

In [2]:
from refinement.refinement_model import RefinementModel
from generation.gpt import GPT
from critique import IsabelleCritique
from critique import CoherenceCritique
from critique import ParsimonyCritique
from critique import UncertaintyCritique
import yaml

from transformers.utils import logging
logging.set_verbosity(logging.CRITICAL)

# data name is default to example_data
# premise and explanation are default to None
data_name = 'esnli_0'
premise = "A couple playing with a little boy on the beach."
hypothesis = "A couple are playing with a young child outside."
explanation = "little boy is a young child."

with open('config.yaml', 'r') as file:
     config = yaml.safe_load(file)
     api_key = config.get('gpt-4o', {}).get('api_key')

llm = GPT('gpt-4o', api_key)

# hard critique model
isabelle_solver = IsabelleCritique(
    generative_model=llm,
    isabelle_session = 'HOL',
    theory_name = data_name
)

# soft critique model
coherence = CoherenceCritique()
parsimony = ParsimonyCritique()
uncertainty = UncertaintyCritique()

all_critique_models = {
    'hard': [isabelle_solver],
    'soft': [coherence, parsimony, uncertainty]
}

critique_mode = 'hard'
prompt_dict = {
    'generate explanation': 'get_explanation_prompt.txt',
    'refine no premise': 'refine_hard_no_premise_prompt.txt',
    'refine with premise': 'refine_hard_with_premise_prompt.txt'
}

refinement_model = RefinementModel(
    generative_model=llm,
    critique_models=all_critique_models,
    critique_mode=critique_mode,
    prompt_dict=prompt_dict
)


results = refinement_model.refine(
    hypothesis = hypothesis,
    premise = premise,
    explanation = explanation,
    data_name = data_name,
    iterations = 2
)

isabelle_solver.shutdown()

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:862: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(



================= Iteration Number 0 ==================

Premise: A couple playing with a little boy on the beach.
Hypothesis: A couple are playing with a young child outside.
Explanation: 
1. little boy is a young child

-------------- Verification and Refinement -------------------

------------------ 1) Syntactic Parsing ------------------

Hypothesis Sentence: 
1. A couple are playing with a young child outside
Subject: A couple
Verb Phrase: are playing with a young child outside
- Main Verb: playing
- Auxiliary Verb: are
Prepositional Phrase (Accompaniment): with a young child
Adverbial Modifier (Location): outside

Explanation Sentence: 
1. little boy is a young child
Subject: little boy
Linking Verb: is
Subject Complement: a young child

Premise Sentence: 
1. A couple playing with a little boy on the beach
Subject: A couple
Verb Phrase: playing with a little boy on the beach
- Main Verb: playing
Prepositional Phrase (Accompaniment): with a little boy
Prepositional Phrase (Locat

## Soft Critique with GPT-4o

In [3]:
from refinement.refinement_model import RefinementModel
from generation.gpt import GPT
from critique import IsabelleCritique
from critique import CoherenceCritique
from critique import ParsimonyCritique
from critique import UncertaintyCritique
import yaml

from transformers.utils import logging
logging.set_verbosity(logging.CRITICAL)

# data name is default to example_data
# premise and explanation are default to None
data_name = 'example_data'
premise = "I blew into the baloon."
hypothesis = "The balloon expanded."

with open('config.yaml', 'r') as file:
     config = yaml.safe_load(file)
     api_key = config.get('gpt-4o', {}).get('api_key')

llm = GPT('gpt-4o', api_key)

# hard critique model
isabelle_solver = IsabelleCritique(
    generative_model=llm,
    isabelle_session = 'HOL',
    theory_name = data_name
)

# soft critique model
coherence = CoherenceCritique()
parsimony = ParsimonyCritique()
uncertainty = UncertaintyCritique()

all_critique_models = {
    'hard': [isabelle_solver],
    'soft': [coherence, parsimony, uncertainty]
}

critique_mode = 'soft'
prompt_dict = {
    'generate explanation': 'generate_explanation_prompt.txt',
    'refine no premise': 'refine_soft_no_premise_prompt.txt',
    'refine with premise': 'refine_soft_with_premise_prompt.txt'
}

refinement_model = RefinementModel(
    generative_model=llm,
    critique_models=all_critique_models,
    critique_mode=critique_mode,
    prompt_dict=prompt_dict
)

results = refinement_model.refine(
    hypothesis = hypothesis,
    premise = premise,
    data_name = data_name,
    iterations = 2
)

isabelle_solver.shutdown()

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:862: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(



================= Iteration Number 0 ==================

Premise: I blew into the baloon.
Hypothesis: The balloon expanded.
Explanation: 
1. Step 1: IF someone blows air into a balloon, THEN the balloon will fill with air
2. Assumption: Blowing air into a balloon introduces air into the balloon, causing it to fill
3. Step 2: IF a balloon fills with air, THEN it will expand
4. Assumption: Balloons are made of elastic material that stretches and expands when air is added
5. Step 3: Therefore, since you blew into the balloon, air was introduced into it, causing the balloon to fill with air and expand

-------------- Verification and Refinement -------------------
0 iteration: {'coherence': 0.7487965868785977}
0 iteration: {'parsimony': 1}
0 iteration: {'uncertainty': 1.0020124912261963}

-------------- 4) Results -------------------

0 iteration: [{'coherence': 0.7487965868785977}, {'parsimony': 1}, {'uncertainty': 1.0020124912261963}]

-------------- 5) Refinement Feedback -------------

## Soft Critique with Causal LLM

In [2]:
from refinement.refinement_model import RefinementModel
from generation.causal import CausalLM
from critique import IsabelleCritique
from critique import CoherenceCritique
from critique import ParsimonyCritique
from critique import UncertaintyCritique
import yaml

from transformers.utils import logging
logging.set_verbosity(logging.CRITICAL)

# data name is default to example_data
# premise and explanation are default to None
data_name = 'example_data'
premise = "I blew into the baloon."
hypothesis = "The balloon expanded."

with open('config.yaml', 'r') as file:
     config = yaml.safe_load(file)
     api_key = config.get('huggingface', {}).get('api_key')

llm = CausalLM('meta-llama/Llama-3.1-8B-Instruct', api_key)

# hard critique model
isabelle_solver = IsabelleCritique(
    generative_model=llm,
    isabelle_session = 'HOL',
    theory_name = data_name
)

# soft critique model
coherence = CoherenceCritique()
parsimony = ParsimonyCritique()
uncertainty = UncertaintyCritique()

all_critique_models = {
    'hard': [isabelle_solver],
    'soft': [coherence, parsimony, uncertainty]
}

critique_mode = 'soft'
prompt_dict = {
    'generate explanation': 'generate_explanation_prompt.txt',
    'refine no premise': 'refine_soft_no_premise_prompt.txt',
    'refine with premise': 'refine_soft_with_premise_prompt.txt'
}

refinement_model = RefinementModel(
    generative_model=llm,
    critique_models=all_critique_models,
    critique_mode=critique_mode,
    prompt_dict=prompt_dict
)

results = refinement_model.refine(
    hypothesis = hypothesis,
    premise = premise,
    data_name = data_name,
    iterations = 2
)

isabelle_solver.shutdown()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:862: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(



================= Iteration Number 0 ==================

Premise: I blew into the baloon.
Hypothesis: The balloon expanded.
Explanation: 
1. Let's break down the test example step-by-step
2. Step 1: IF air is blown into a balloon, THEN the balloon will expand
3. Assumption: When air is introduced into a balloon, it increases the internal pressure, causing the balloon to expand
4. Step 2: IF the balloon expands, THEN it will increase in size
5. Assumption: The expansion of air within the balloon results in an increase in its physical dimensions
6. Step 3: IF the balloon increases in size, THEN it will become larger than its original size
7. Assumption: Expansion implies an increase in size, making the balloon larger than it was before
8. Step 4: IF the balloon becomes larger than its original size, THEN it can be said to have expanded
9. Assumption: Expansion is a change in size, and if the balloon becomes larger, it can be described as expanded
10. Step 5: Therefore, since I blew air 